In [1]:
import numpy as np 
import pandas as pd 
import lightgbm as lgb

from sklearn.model_selection import train_test_split, StratifiedKFold
#from sklearn.metrics import cohen_kappa_score, accuracy_score,balanced_accuracy_score
from sklearn.utils import shuffle
from sklearn.model_selection import KFold # Use KFold for regression
from sklearn.metrics import mean_squared_error


#from plotly import express as px

#from utils import plot_confusion_matrix

import os

import optuna
from optuna.artifacts import FileSystemArtifactStore, upload_artifact

from joblib import load, dump
import pyarrow.parquet as pq


import gc


c:\Users\agust\anaconda3\envs\FAA\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Armado Dataset

In [2]:
# Paths
BASE_DIR = 'C:/Users/agust/Documents/MaestriaCD/LAB3/final/'

#PATH_TO_TRAIN = os.path.join(BASE_DIR, "Dataset/train.parquet")
PATH_TO_TRAIN = os.path.join(BASE_DIR, "train_producto.parquet")
PATH_TO_MODELS = os.path.join(BASE_DIR, "LGBM2/models")
PATH_TO_TEMP_FILES = os.path.join(BASE_DIR, "LGBM2/optuna_/optuna_temp_artifacts")
PATH_TO_OPTUNA_ARTIFACTS = os.path.join(BASE_DIR, "LGBM2/optuna_/optuna_artifacts")
PATH_TO_MODELS_OLD=PATH_TO_MODELS_OLD = os.path.join(BASE_DIR, "LGBM2/models/old")

SEED = 42
BATCH_SIZE = 50
TEST_SIZE = 0.2
PATH_TO_TRAIN

'C:/Users/agust/Documents/MaestriaCD/LAB3/final/train_producto.parquet'

In [3]:
dataset = pd.read_parquet(PATH_TO_TRAIN)

In [4]:
dataset.shape

(29652, 280)

In [5]:
char_feats = dataset.select_dtypes(include=['object', 'category']).columns.tolist()
print(char_feats)
#numeric_feats = [f for f in dataset.columns if dataset[f].dtype!='O']

['cat1', 'cat2', 'cat3', 'brand', 'estado_producto']


In [6]:
numeric_feats = dataset.select_dtypes(include=['number']).columns.tolist()
print(numeric_feats)

['periodo', 'product_id', 'tn', 'avg_tn', 'std_tn', 'clientes_distintos', 'plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'inicio_vida_p', 'fin_vida_p', 'sku_size', 'stock_final', 'dias_mes_4', 'anio', 'mes', 'trimestre', 'outlier', 'outlier-2', 'tn_lag_1', 'tn_lag_2', 'tn_lag_3', 'tn_lag_4', 'tn_lag_5', 'tn_lag_6', 'tn_lag_7', 'tn_lag_8', 'tn_lag_9', 'tn_lag_10', 'tn_lag_11', 'tn_lag_12', 'tn_lag_13', 'tn_lag_14', 'tn_lag_15', 'tn_lag_16', 'tn_lag_17', 'tn_lag_18', 'tn_lag_19', 'tn_lag_20', 'tn_lag_21', 'tn_lag_22', 'tn_lag_23', 'tn_lag_24', 'tn_lag_25', 'tn_lag_26', 'tn_lag_27', 'tn_lag_28', 'tn_lag_29', 'tn_lag_30', 'tn_lag_31', 'tn_lag_32', 'tn_lag_33', 'tn_lag_34', 'tn_lag_35', 'tn_lag_36', 'tn_media_movil_3', 'tn_media_movil_6', 'tn_media_movil_12', 'tn_std_movil_3', 'tn_std_movil_6', 'tn_std_movil_12', 'tn_min_movil_3', 'tn_min_movil_6', 'tn_min_movil_12', 'delta1_media_movil_12', 'delta1_media_movil_3', 'delta1_media_movil_6', 'delta_tn_1', 'delta_tn_2', 'delta_

In [7]:
dataset.shape

(29652, 280)

In [8]:
gc.collect()

20

In [9]:
def reduce_mem_usage(df, verbose=True):
    start_mem = df.memory_usage(deep=True).sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtype

        # Solo nos ocupamos de numéricos
        if not pd.api.types.is_numeric_dtype(col_type):
            if not pd.api.types.is_categorical_dtype(col_type):
                df[col] = df[col].astype('category')
            continue

        c_min, c_max = df[col].min(), df[col].max()
        has_na = df[col].isnull().any()

        # --- ENTEROS ---
        if pd.api.types.is_integer_dtype(col_type):
            # 1) Sin nulos → numpy ints
            if not has_na:
                if c_min >= np.iinfo(np.int8).min  and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)


        # --- FLOTANTES ---
        else:
            df[col] = df[col].astype(np.float32)

    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose:
        print(f'Uso de memoria inicial del DataFrame: {start_mem:.2f} MB')
        print(f'Uso de memoria final del DataFrame:   {end_mem:.2f} MB')
        print(f'Memoria reducida en un {(100*(start_mem-end_mem)/start_mem):.2f}%')
    return df

In [10]:
dataset = reduce_mem_usage(dataset)

C:\Users\agust\AppData\Local\Temp\ipykernel_11468\290016523.py:9: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(col_type):


Uso de memoria inicial del DataFrame: 17.24 MB
Uso de memoria final del DataFrame:   30.27 MB
Memoria reducida en un -75.64%


C:\Users\agust\AppData\Local\Temp\ipykernel_11468\290016523.py:9: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(col_type):


In [11]:
gc.collect()

0

In [12]:
dataset.shape

(29652, 280)

In [13]:
train, test = train_test_split(dataset,
                               test_size = TEST_SIZE,
                               random_state = SEED,
                               ) #stratify = dataset.clase saco esto porque no es un problema de clasificacion

In [14]:
train.shape

(23721, 280)

In [15]:
# Eliminar el DataFrame
del dataset

gc.collect()

0

In [16]:
#train = reduce_mem_usage(train)
#gc.collect()

In [17]:
#test = reduce_mem_usage(test)
#gc.collect()

In [18]:
label = 'clase'
features = [col for col in train.columns if col != label]

In [19]:
train.to_parquet("train_tmp.parquet", index=False)
del train
gc.collect()

train = pd.read_parquet("train_tmp.parquet")


In [20]:
test.to_parquet("test_tmp.parquet", index=False)
del test
gc.collect()

test = pd.read_parquet("test_tmp.parquet")

In [21]:
train.shape

(23721, 280)

In [22]:
test.shape

(5931, 280)

In [23]:
X_train = train[features]
y_train = train[label]

X_test = test[features]
y_test = test[label]

In [24]:
X_train.shape

(23721, 279)

In [25]:
del train
#del test

gc.collect()

0

In [26]:
print("X_train shape:", X_train.shape)

X_train shape: (23721, 279)


In [27]:
#X_train = reduce_mem_usage(X_train)
#gc.collect()


In [28]:
#y_train = reduce_mem_usage(y_train)
#gc.collect()

In [29]:
#X_test = reduce_mem_usage(X_test)
#gc.collect()

In [30]:
#y_test = reduce_mem_usage(y_test)
#gc.collect()

## Entrenamiento

In [31]:
def lgb_objective(trial):

    # Parámetros para LightGBM
    lgb_params = {
        'objective': 'regression',
        'metric': 'mse', # Cambiado a Mean Squared Error
        'verbosity': -1,
        # 'num_class': len(y_train.unique()), # Eliminar, esto es para clasificación
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }

    # Voy a generar estimaciones de los 5 modelos del CV sobre los datos test y los acumulo en la matriz scores_ensemble
    # Para regresión, scores_ensemble será un array 1D
    scores_ensemble = np.zeros(len(y_test),dtype=np.float32)

    # Score del 5 fold CV inicializado en 0
    score_folds = 0

    # Numero de splits del CV
    n_splits = 5

    # Objeto para hacer el split de CV (usar KFold para regresión, ya que StratifiedKFold es para clasificación)
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=SEED) # Added shuffle and random_state for reproducibility

    for i, (if_index, oof_index) in enumerate(kf.split(X_train, y_train)): # Usar kf.split

        # Dataset in fold (donde entreno)
        lgb_if_dataset = lgb.Dataset(data=X_train.iloc[if_index],
                                     label=y_train.iloc[if_index],
                                     free_raw_data=True, categorical_feature=char_feats) #cambie free raw data a true

        # Dataset Out of fold (donde mido la performance del CV)
        lgb_oof_dataset = lgb.Dataset(data=X_train.iloc[oof_index],
                                      label=y_train.iloc[oof_index],
                                      free_raw_data=True, categorical_feature=char_feats) #cambie free raw data a true

        # Entreno el modelo
        lgb_model = lgb.train(lgb_params,
                              lgb_if_dataset,
                              valid_sets=lgb_oof_dataset,
                              callbacks=[lgb.early_stopping(50, verbose=False)],
                              # feval = mean_squared_error(y_test, preds, squared=False) # Eliminar o definir correctamente para custom metric
                             )

        # Acumulo las predicciones continuas para el conjunto de test
        scores_ensemble = scores_ensemble + lgb_model.predict(X_test)

        # Score del fold (registros de dataset train que en este fold quedan out of fold)
        # Calcular MSE para el fold OOF
        oof_preds = lgb_model.predict(X_train.iloc[oof_index])
        score_folds += mean_squared_error(y_train.iloc[oof_index], oof_preds) / n_splits
        
        # ⬇️ Liberar memoria de objetos pesados del fold
        del lgb_model, lgb_if_dataset, lgb_oof_dataset, oof_preds
        gc.collect()
        print(f"fin split {i}")

    # Promedio las predicciones del ensemble para el conjunto de test
    scores_ensemble = scores_ensemble / n_splits

    # Guardo prediccion del trial sobre el conjunto de test
    # Genero nombre de archivo
    #predicted_filename = os.path.join(PATH_TO_TEMP_FILES,f'test_{trial.study.study_name}_{trial.number}.joblib')
    # Copia del dataset para guardar la prediccion
    #predicted_df = test.copy()
    #predicted_df= pd.DataFrame({
    #'periodo': test['periodo'],
    #'customer_id': test['customer_id'],  # o el ID que necesites
    #'product_id' : test['product_id'],
    #'pred': scores_ensemble
    #})
    
    # Genero columna pred con predicciones promediadas de los 5 folds
    #predicted_df['pred'] = scores_ensemble
    # Grabo dataframe en temp_artifacts
    #dump(predicted_df, predicted_filename)
    # Indico a optuna que asocie el archivo generado al trial
    #upload_artifact(trial, predicted_filename, artifact_store)
    #del predicted_df
    gc.collect()
    
    # Grabo métricas de regresión en lugar de matriz de confusión
    # Puedes guardar métricas como MSE, RMSE, MAE, R2, etc. en un archivo de texto o log
    # Por ejemplo, calcular el MSE en el conjunto de test
    test_mse = mean_squared_error(y_test, scores_ensemble)
    trial.set_user_attr("test_mse", test_mse) # Almacenar MSE del test como atributo de usuario

    del scores_ensemble
    gc.collect()

    # Si quieres una visualización, podrías generar un scatter plot de predicciones vs valores reales
    # O un histograma de residuos. Aquí un placeholder para un "regression_metrics.txt"
    #regression_metrics_filename = os.path.join(PATH_TO_TEMP_FILES, f'regression_metrics_{trial.study.study_name}_{trial.number}.txt')
    #with open(regression_metrics_filename, 'w') as f:
    #    f.write(f"Test Mean Squared Error: {test_mse}\n")
    #    # Add other regression metrics if desired
    #upload_artifact(trial, regression_metrics_filename, artifact_store)
    
    gc.collect()

    # Devuelvo el score promedio de MSE del 5-fold CV a Optuna para que optimice en base a eso
    # Optuna minimiza por defecto, por lo que devolver MSE es apropiado.
    return score_folds

In [32]:
#Inicio el store de artefactos (archivos) de optuna
artifact_store = FileSystemArtifactStore(base_path=PATH_TO_OPTUNA_ARTIFACTS)

#Genero estudio
study = optuna.create_study(direction='minimize',
                            storage="sqlite:///db.sqlite2",  # Specify the storage URL here.
                            study_name="Producto_base",
                            load_if_exists = True)
#Corro la optimizacion
study.optimize(lgb_objective, n_trials=1000, gc_after_trial=True)

[I 2025-07-18 23:48:40,457] A new study created in RDB with name: Producto_base


fin split 0
fin split 1
fin split 2
fin split 3
fin split 4


[I 2025-07-18 23:49:06,387] Trial 0 finished with value: 1006.8424965183314 and parameters: {'lambda_l1': 6.806000143792703e-06, 'lambda_l2': 0.34981272690062015, 'num_leaves': 84, 'feature_fraction': 0.4042366493175729, 'bagging_fraction': 0.6514142861366973, 'bagging_freq': 4, 'min_child_samples': 78}. Best is trial 0 with value: 1006.8424965183314.


fin split 0
fin split 1
fin split 2
fin split 3
fin split 4


[I 2025-07-18 23:49:18,215] Trial 1 finished with value: 995.389801595706 and parameters: {'lambda_l1': 4.2053691520507865e-08, 'lambda_l2': 0.010093542520468502, 'num_leaves': 14, 'feature_fraction': 0.8019814964856453, 'bagging_fraction': 0.9279469076135621, 'bagging_freq': 6, 'min_child_samples': 84}. Best is trial 1 with value: 995.389801595706.


...

[I 2025-07-19 15:42:33,743] Trial 997 finished with value: 822.7514838372526 and parameters: {'lambda_l1': 0.007429856782085946, 'lambda_l2': 0.3902779946920211, 'num_leaves': 30, 'feature_fraction': 0.9818604217933194, 'bagging_fraction': 0.9638952409304622, 'bagging_freq': 1, 'min_child_samples': 6}. Best is trial 594 with value: 797.0830964467458.


fin split 0
fin split 1
fin split 2
fin split 3
fin split 4


[I 2025-07-19 15:42:54,584] Trial 998 finished with value: 799.7680486086556 and parameters: {'lambda_l1': 0.0010140964025127676, 'lambda_l2': 0.21431602291306248, 'num_leaves': 35, 'feature_fraction': 0.9591847504317307, 'bagging_fraction': 0.9738797893630373, 'bagging_freq': 1, 'min_child_samples': 5}. Best is trial 594 with value: 797.0830964467458.


fin split 0
fin split 1
fin split 2
fin split 3
fin split 4


[I 2025-07-19 15:43:12,660] Trial 999 finished with value: 856.8821491203046 and parameters: {'lambda_l1': 0.0007824856441671556, 'lambda_l2': 0.10830979984339686, 'num_leaves': 27, 'feature_fraction': 0.9299608360893791, 'bagging_fraction': 0.9645607308038875, 'bagging_freq': 1, 'min_child_samples': 5}. Best is trial 594 with value: 797.0830964467458.


## Guardar modelos con mejores parametros

In [33]:
# Buscar el trial con menor mse_test en los atributos de usuario
best_trial = min(
    [t for t in study.trials if t.user_attrs.get("test_mse") is not None],
    key=lambda t: t.user_attrs["test_mse"]
)

In [34]:
print(best_trial)

FrozenTrial(number=204, state=1, values=[851.5274412416657], datetime_start=datetime.datetime(2025, 7, 19, 1, 4, 23, 814356), datetime_complete=datetime.datetime(2025, 7, 19, 1, 4, 36, 858936), params={'lambda_l1': 1.9312102498602122e-05, 'lambda_l2': 0.004066073706786463, 'num_leaves': 36, 'feature_fraction': 0.400418034256639, 'bagging_fraction': 0.9660851334117331, 'bagging_freq': 5, 'min_child_samples': 14}, user_attrs={'test_mse': 736.68991952207}, system_attrs={}, intermediate_values={}, distributions={'lambda_l1': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'lambda_l2': FloatDistribution(high=10.0, log=True, low=1e-08, step=None), 'num_leaves': IntDistribution(high=256, log=False, low=2, step=1), 'feature_fraction': FloatDistribution(high=1.0, log=False, low=0.4, step=None), 'bagging_fraction': FloatDistribution(high=1.0, log=False, low=0.6, step=None), 'bagging_freq': IntDistribution(high=7, log=False, low=1, step=1), 'min_child_samples': IntDistribution(high=

In [35]:
best_params = best_trial.params.copy()
best_params.update({
    "objective": "regression",#los parametros que no se optimizan (los fijos) hay que escribirlos
    "metric": "mse",
    "verbosity": -1,
    'random_state': SEED
    
})

In [36]:
print( best_trial.params)

{'lambda_l1': 1.9312102498602122e-05, 'lambda_l2': 0.004066073706786463, 'num_leaves': 36, 'feature_fraction': 0.400418034256639, 'bagging_fraction': 0.9660851334117331, 'bagging_freq': 5, 'min_child_samples': 14}


In [37]:
def entrenar_lgbm_final_con_cv(base_params, seed):
    """
    Realiza una evaluación CV y entrena un modelo final para una SEMILLA específica.
    
    Args:
        base_params (dict): Hiperparámetros base optimizados por Optuna.
        seed (int): La semilla para esta ejecución específica (usada en KFold y en el modelo).
        
    Returns:
        tuple: (mse_cv_promedio, mse_test)
    """
    
    # Creamos el diccionario completo de parámetros para esta ejecución
    run_params = base_params.copy()
    run_params.update({
        'random_state': seed  # <--- CORRECCIÓN: Usamos la semilla de esta ejecución
    })

    # --- Evaluación con CV ---
    print(f"\n{'-'*15} Iniciando CV para SEED = {seed} {'-'*15}")
    score_folds = 0
    best_iterations = [] # <--- CORRECCIÓN 3: Lista para guardar las mejores iteraciones
    n_splits = 5
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)

    for fold, (if_index, oof_index) in enumerate(kf.split(X_train, y_train)):
        print(f"🔁 Fold {fold + 1}")

        lgb_if_dataset = lgb.Dataset(data=X_train.iloc[if_index], label=y_train.iloc[if_index], free_raw_data=False, categorical_feature=char_feats)
        lgb_oof_dataset = lgb.Dataset(data=X_train.iloc[oof_index], label=y_train.iloc[oof_index], free_raw_data=False, categorical_feature=char_feats)

        model_cv = lgb.train(
            run_params,
            lgb_if_dataset,
            valid_sets=lgb_oof_dataset,
            callbacks=[lgb.early_stopping(50, verbose=False)]
        )

        best_iterations.append(model_cv.best_iteration) # <--- CORRECCIÓN 3: Guardamos la iteración
        
        oof_preds = model_cv.predict(X_train.iloc[oof_index])
        mse = mean_squared_error(y_train.iloc[oof_index], oof_preds)
        score_folds += mse / n_splits

    print(f"📊 MSE promedio 5-Fold CV: {score_folds:.6f}")

    # --- Entrenamiento final en todo el dataset ---
    print("\n🚀 Entrenando modelo final...")
    
    # <--- CORRECCIÓN 3: Usamos el promedio de iteraciones para el modelo final
    final_num_boost_round = int(np.mean(best_iterations))
    print(f"🎯 Usando un promedio de {final_num_boost_round} rondas (promedio de: {best_iterations})")

    final_dataset = lgb.Dataset(data=X_train, label=y_train, free_raw_data=True, categorical_feature=char_feats)
    final_model = lgb.train(
        run_params,
        final_dataset,
        num_boost_round=final_num_boost_round
    )
    
    # Guardar el modelo final (opcionalmente, nombrar con la semilla)
    model_path = os.path.join(PATH_TO_MODELS, f"lgb_model_seed_{seed}.joblib")
    dump(final_model, model_path)
    print(f"✅ Modelo final guardado en: {model_path}")
    
    # --- Evaluación en test ---
    y_pred_test = final_model.predict(X_test)
    test_mse = mean_squared_error(y_test, y_pred_test)
    print(f"📉 MSE en test: {test_mse:.6f}")
    
    gc.collect()

    return score_folds, test_mse

In [38]:
# Mover modelos previos antes del loop
for fname in os.listdir(PATH_TO_MODELS):
    if fname.endswith('.joblib'):
        src_path = os.path.join(PATH_TO_MODELS, fname)
        dst_path = os.path.join(PATH_TO_MODELS_OLD, fname)
        shutil.move(src_path, dst_path)
        print(f"🔁 Modelo movido a backup: {fname}")

In [39]:
#results = []
SEEDS = [5,915,15,666,9999,37,45,125,90,1000,42,3,753,159,852,10,7,1,1050,654]

for current_seed in SEEDS:
    # <--- CORRECCIÓN 1 y 2: La función ahora se llama con los parámetros base y la semilla actual
    mse_cv, mse_test = entrenar_lgbm_final_con_cv(
        base_params=best_params, 
        seed=current_seed
    )


--------------- Iniciando CV para SEED = 5 ---------------
🔁 Fold 1
🔁 Fold 2
🔁 Fold 3
🔁 Fold 4
🔁 Fold 5
📊 MSE promedio 5-Fold CV: 843.933479

🚀 Entrenando modelo final...
🎯 Usando un promedio de 96 rondas (promedio de: [96, 96, 100, 98, 94])
✅ Modelo final guardado en: C:/Users/agust/Documents/MaestriaCD/LAB3/final/LGBM2/models\lgb_model_seed_5.joblib
📉 MSE en test: 820.240645

--------------- Iniciando CV para SEED = 915 ---------------
🔁 Fold 1
🔁 Fold 2
🔁 Fold 3
🔁 Fold 4
🔁 Fold 5
📊 MSE promedio 5-Fold CV: 922.615334

🚀 Entrenando modelo final...
🎯 Usando un promedio de 98 rondas (promedio de: [100, 98, 99, 95, 99])
✅ Modelo final guardado en: C:/Users/agust/Documents/MaestriaCD/LAB3/final/LGBM2/models\lgb_model_seed_915.joblib
📉 MSE en test: 747.321087

--------------- Iniciando CV para SEED = 15 ---------------
🔁 Fold 1
🔁 Fold 2
🔁 Fold 3
🔁 Fold 4
🔁 Fold 5
📊 MSE promedio 5-Fold CV: 885.865016

🚀 Entrenando modelo final...
🎯 Usando un promedio de 96 rondas (promedio de: [97, 89, 100,

In [40]:
import joblib

# Import the LGBMEnsemble class definition before loading the model
# from lgbm_ensemble import LGBMEnsemble  # Uncomment and adjust if you have the class in a module

# ...existing code...

#model = joblib.load( 'C:/Users/agust/Documents/MaestriaCD/LAB3/final/LGBM/models/lgb_model_seed_654.joblib')


In [41]:
#type(model)


In [42]:
# Obtener la importancia de cada feature
#importancia = model.feature_importance(importance_type='gain')  # o 'split'


In [43]:
#feature_names = model.feature_name()


In [44]:
import pandas as pd
import matplotlib.pyplot as plt

#feature_names = model.feature_name()
#importancias = model.feature_importance(importance_type='gain')

#df_importancias = pd.DataFrame({
#    'Feature': feature_names,
#    'Importance': importancias
#}).sort_values(by='Importance', ascending=False)

#print(df_importancias)


In [45]:
#df_importancias.head(20).plot(kind='barh', x='Feature', y='Importance', figsize=(10, 6), legend=False)
#plt.gca().invert_yaxis()
#plt.title('LightGBM Feature Importance (gain)')
#plt.tight_layout()
#plt.show()
